<a href="https://colab.research.google.com/github/prasann25/colab/blob/main/08_introduction_to_nlp_in_tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction to NLP Fundamentals in TensorFlow

NLP has the goal of deriving information out of natural language(could be sequences text or speech).
Another common term for NLP problems is sequence to sequence problems(seq2seq).

## Check for GPU 

In [1]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-b470b110-2222-4c7d-cb00-cea7e71131d5)


## Get helper functions

In [3]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

# Import series of helper functions for the notebook
from helper_functions import unzip_data, create_tensorboard_callback, compare_historys, plot_loss_curves

--2021-08-04 00:35:42--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py.1’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2021-08-04 00:35:43 (112 MB/s) - ‘helper_functions.py.1’ saved [10246/10246]



## Get a text dataset

The dataset we're going to be using is Kaggle's introduction to NLP dataset(text samples of Tweets labelled as disaster or not disaster).

See the original source here : https://www.kaggle.com/c/nlp-getting-started

In [4]:
!wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip

--2021-08-04 00:38:04--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.101.128, 142.250.141.128, 142.251.2.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.101.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.004s  

2021-08-04 00:38:05 (142 MB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



In [7]:
# Unzip data
unzip_data("nlp_getting_started.zip")

## Visualizing a text dataset

To visualize our text samples, we first have to read them in , one way to do sou would be to use Python : https://realpython.com/python-csv/

But I prefer to get visual straight away.
So another way to do is to use pandas...



In [9]:
import pandas as pd

train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [14]:
# Shuffle training dataframe

train_df_shuffled = train_df.sample(frac=1, random_state=42)
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [15]:
# What does the test dataframe look like ?
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [16]:
# How many examples of each class?

train_df.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

 > **Note** : If your data is not balanced, there are some things you can do for imbalanced dataset
https://www.tensorflow.org/tutorials/structured_data/imbalanced_data

In [17]:
# How many total samples ?
len(train_df), len(test_df)

(7613, 3263)

In [19]:
import random
random_index = random.randint(0, len(train_df)-5)
random_index

6921

In [23]:
# Let's visualize some random training examples
import random
random_index = random.randint(0, len(train_df)-5) # create random indexes not higher than total number of samples
for row in train_df_shuffled[["text", "target"]][random_index:random_index+5].itertuples() :
  _,text,target = row
  print(f"Target: {target}", "(real disaster)" if target > 0 else "(not real disaster)")
  print(f"Text:\n{text}\n")
  print("---\n")


Target: 0 (not real disaster)
Text:
Best windows torrent client? was recommended Deluge but it looks like it was written 10 years ago with java swing and 'uses' worse

---

Target: 1 (real disaster)
Text:
California is battling its scariest 2015 wildfire so far. http://t.co/Lec1vmS7x2

---

Target: 1 (real disaster)
Text:
mentions of 'theatre +shooting' on Twitter spike 30min prior to $ckec collapse http://t.co/uuBOvy9GQI

---

Target: 0 (not real disaster)
Text:
trapped in its disappearance

---

Target: 1 (real disaster)
Text:
#Saudi Arabia: #Abha: Fatalities reported following suicide bombing at mosque; avoid area http://t.co/1xW0Z8ZeqW

---

